### 说明：主要用于尝试运行评价模型；并进一步尝试训练N20产生器。

## 该部分来自于CRISPRon和CRISPRspec_CRISPRoff, 使用时请加引用。

# 环境配置和函数定义

In [1]:
import sys
import os
import glob
from collections import OrderedDict 
import numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow.keras import models
from Bio import SeqIO
import pandas as pd
import pickle, subprocess, gzip
from math import exp

## Components of CRISPRspec_CRISPRoff 

In [2]:
##############################################################
#################### Energy computation #########################
##############################################################

ENERGY_MODELS_PICKLE_FILE = "energy_dics.pkl"

RI_REV_NT_MAP = {'-':'', 'a':'T', 'A':'T', 'c':'G', 'C':'G', 'g':'C', 'G':'C',
              't':'A', 'T':'A', 'u':'A', 'U':'A', 'n':'N', 'N':'N'}

RI_DNA_DNA_NN={'AA':{'TT':-1.00}, 'TT':{'AA':-1.00}, 'AT':{'TA':-0.88}, 'TA':{'AT':-0.58},
            'CA':{'GT':-1.45}, 'TG':{'AC':-1.45}, 'GT':{'CA':-1.44}, 'AC':{'TG':-1.44},
            'CT':{'GA':-1.28}, 'AG':{'TC':-1.28}, 'GA':{'CT':-1.30}, 'TC':{'AG':-1.30},
            'CG':{'GC':-2.17}, 'GC':{'CG':-2.24}, 'GG':{'CC':-1.84}, 'CC':{'GG':-1.84}}

RI_MATCH_noGU = {'A':{'A':False, 'C':False, 'G':False, 'T':True},
         'C':{'A':False, 'C':False, 'G':True, 'T':False},
         'G':{'A':False, 'C':True, 'G':False, 'T':False},
         'T':{'A':True, 'C':False, 'G':False, 'T':False}}

########## READ THE 2mer 3mer 4mer energies ###############
RNA_DNA_internal_loop ={3:3.2, 4:3.555, 5:3.725, 6:3.975, 7:4.16, 8:4.33, 9:4.495, 10:4.6, 11:4.7}
RNA_DNA = None
def read_energy_parameters(ENERGY_MODELS_PICKLE_FILE = "energy_dics.pkl"):
    global RNA_DNA
    energy_reader=open(ENERGY_MODELS_PICKLE_FILE, 'rb')
    RNA_DNA = pickle.load(energy_reader)
    energy_reader.close()

####### Necessary for self-folding ######
RNAFOLD_EXE = "RNAfold"

######### positional energy contribution weights ###################
# LAST weight is filled but not used, (Left-over from some experimental option)
POS_WGH=[1.80067099242007, 1.95666668400006, 1.90472004401173, 2.13047270152512, 1.37853848098249, 1.46460783730408, 1.0, 1.387220146823, 1.51401000729362, 1.98058344620751, 1.87939168587699, 1.7222593588838, 2.02228445489326, 1.92692086621503, 2.08041972716723, 1.94496755678903, 2.14539112893591, 2.04277109036766, 2.24911493451185, 2.25]

# LAST weight is filled but not used, (Left-over from some experimental option)
DNA_POS_WGH = [1.22245576981774, 1.24561578622024, 1.37883177517399, 1.39146340276523, 1.24308180746857, 1.09598194424544, 1.0, 1.11695025382169, 1.11589045394936, 1.22243614188218, 1.21317477033274, 1.07125942316357, 1.25205871414019, 1.21445408158483, 1.20971491326295, 1.21076785001579, 1.2480898972246, 1.40301355270318, 1.41221084925493, 1.4]

######### pam correction parameters for pam-updated energy ##########
pam_ratios={"GGG":1.0, "AGG":1.0, "CGG":1.0, "TGG":1.0, "GAG":0.9, "AAG":0.9, "CAG":0.9, "TAG":0.9, "GGA":0.8, "AGA":0.8, "CGA":0.8, "TGA":0.8, "OTHERS": 0.0}
pam_ratio_count = 3

########## gRNA folding #######################
grna_folding_engs = {}
def get_rnafold_eng(seq, rid="temp_grna_id"):
    if seq not in grna_folding_engs:
        no_constraint_eng = None
        l = len(seq)
        cmd = RNAFOLD_EXE
        p = subprocess.Popen([cmd, '--noPS'], shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True).communicate(input=">"+rid+"\n"+seq+"\n\n")
        if p[1]=="":
            no_constraint_eng = float(p[0].rstrip().split()[-1].replace('(','').replace(')',''))
        else:
            sys.stderr.write("#ERROR:RNAfold run went wrong: "+cmd+"; "+p[1]+"\n")
            exit()
        grna_folding_engs[seq] = no_constraint_eng
    return grna_folding_engs[seq]

# Stacking energy is distributed to positions only for interior loops
def calcRNADNAenergy(guideSeq, otSeq, GU_allowed=False):
    guideSeq = guideSeq.upper()[:-3]
    seq = ''.join([RI_REV_NT_MAP[c] for c in otSeq[:-3]])

    spos = -1
    epos = -1

    energy = [0.0]*len(guideSeq)

    MATCH = RI_MATCH_noGU

    for i in range(len(seq)):
        if MATCH[guideSeq[i]][seq[i]]:
            if spos==-1:
                spos = i
            epos = i

    i = spos
    while i<epos:
        j = i+1
        while MATCH[seq[j]][guideSeq[j]]==False:
            j = j+1

            if j > epos:
                break
        if j > epos:
            break

        loop_size = (j-i)-1
        eng_con = 0
        if loop_size<3:
            eng_con = RNA_DNA[loop_size][guideSeq[i:j+1]][seq[i:j+1]]
            # if there is a stack in the beginning or end AU GU penalty is still needed
            if loop_size == 0:
                if (i==spos and (guideSeq[i]=="T" or seq[i]=="T")) or (j==epos and (guideSeq[j]=="T" or seq[j]=="T")):
                    eng_con += 0.25
        else:
            eng_con = float(RNA_DNA_internal_loop[loop_size]) + float(RNA_DNA[0][guideSeq[i:i+2]][seq[i:i+2]]) + float(RNA_DNA[0][guideSeq[j-1:j+1]][seq[j-1:j+1]])

        for k in range(loop_size+1):
            energy[i+k] += eng_con/(loop_size+1)

        i = j

    return energy

################ DNA-DNA opening #############
def calcDNAopeningScore(otSeq):
    seq = otSeq.upper()[:-3]
    energy = [0.0]*len(seq)
    for i in range(1, len(seq)):
        energy[i] = float(RI_DNA_DNA_NN[seq[i-1]+seq[i]][RI_REV_NT_MAP[seq[i-1]]+RI_REV_NT_MAP[seq[i]]])
    return energy
#################################################################################################

REV_NT_MAP = {'-':'', 'a':'T', 'A':'T', 'c':'G', 'C':'G', 'g':'C', 'G':'C', 
              't':'A', 'T':'A', 'u':'A', 'U':'A', 'n':'N', 'N':'N'}

def rev_comp_seq(seq):
    s=''
    for c in seq:
        s =  REV_NT_MAP[c] + s
    return s

def comp_seq(seq):
    s=''
    for c in seq:
        s =  s + REV_NT_MAP[c]
    return s

############# Get interaction energy ##################
#Employ all the necessary computations on the score vector to get the final free energy
def get_eng(grna_seq, off_seq, score_func, GU_allowed=False, pos_weight=False, pam_corr=False, grna_folding=False, dna_opening=False, dna_pos_wgh=False):
    scores = score_func(grna_seq, off_seq, GU_allowed)

    if pos_weight:
        for i in range(len(scores)):
            if i < 21:
                scores[-(i+1)] = POS_WGH[-(i+1)] * scores[-(i+1)]

    off = sum(scores) + 0.0
    off = (-1.0) * off

    if grna_folding:
        off += get_rnafold_eng(grna_seq[:20])


    if dna_opening:
        dna_scores = calcDNAopeningScore(off_seq)
        if dna_pos_wgh:
            for i in range(len(dna_scores)):
                if i < 21:
                    dna_scores[-(i+1)] = DNA_POS_WGH[-(i+1)] * dna_scores[-(i+1)]
        off += sum(dna_scores)

    if pam_corr:
        if off_seq[-pam_ratio_count:] in pam_ratios.keys():
            off = off * pam_ratios[off_seq[-pam_ratio_count:]]
        else:
            off = off * pam_ratios["OTHERS"]

    return off

#################################################################
# TODO: Turn the ontarget and offSeq into class for ease of understanding the code #

## Compute POFF for the given grna and its off-targets ##
PAR_BETA = 1.000000 / (0.001987 * 310.150000)
def compute_CRISPRspec(ontarget, offSeqs, score_func, GU_allowed=False, pos_weight=False, pam_corr=False, grna_folding=False, dna_opening=False, dna_pos_wgh=False, ignored_chromosomes=set()):
    pf = 0.000000
    on = 0.000000
    CRISPRoff_scores = []

    for offSeq in offSeqs:
        offSeq_eng = get_eng(ontarget[0], offSeq[0], score_func,  GU_allowed=GU_allowed, pos_weight=pos_weight, pam_corr=pam_corr, grna_folding=grna_folding, dna_opening=dna_opening, dna_pos_wgh=dna_pos_wgh)
        if offSeq[1] not in ignored_chromosomes:
            pf = pf + exp(PAR_BETA * offSeq_eng)
        else:
            sys.stderr.write("#WARNING: This off-target sequence is ignored when computing CRISPRspec: '"+"|".join([str(x) for x in offSeq])+"'.\n")
        CRISPRoff_scores.append((offSeq, offSeq_eng))

    on_eng = get_eng(ontarget[0], ontarget[0], score_func, GU_allowed=GU_allowed, pos_weight=pos_weight, pam_corr=pam_corr, grna_folding=grna_folding, dna_opening=dna_opening, dna_pos_wgh=dna_pos_wgh)
    on = exp(PAR_BETA * on_eng)

    CRISPRoff_scores.append(((ontarget[0], ontarget[2], ontarget[3], ontarget[4], ontarget[5]), on_eng))
    return (pf/(pf+on)), CRISPRoff_scores

###############################################################
################ Input readers ################################
###############################################################
# Returns all valid Cas9 targets predicted with RIsearch2 in the genome
# Result file must have been generated with -p3 option
# grna is the sequence without the PAM addition.
# If PAM is added last parameter must be set to False
def read_risearch_results(guideSeq, ris_file, noPAM_given=True, count_mms=False, on_targets = [], offSeqs = [], off_counts = {"GG":[0]*7, "AG":[0]*7, "GA":[0]*7}):
    inf = gzip.open(ris_file, 'rt') if ris_file.endswith(".gz") else open(ris_file, 'rt')
    x = 0
    for line in inf:
        x = x + 1
        cols = line.rstrip().split("\t")
        if len(cols)>11 and len(cols[10])>3 and len(cols[11])>0:
            gid, qs, qe, tc, ts, te, tst, en, ist, iseq, pamseq, preseq= cols[:12]
            PAM = comp_seq(pamseq[:3])
            if PAM[1:3] in ["GG", "AG", "GA"]:
                offseq = comp_seq(iseq)
                if tst=="+":
                    ts = str(int(ts)-4)
                    tst="-"
                else:
                    ts = str(int(ts)-1)
                    te = str(int(te)+3)
                    tst="+"

                # determine and save the number of mismatches for this guide
                mm_count = sum ( offseq[i] != guideSeq[i] for i in range(len(offseq)) )
                if count_mms:
                    if mm_count<7:
                        off_counts[PAM[1:3]][mm_count] += 1
                if mm_count<7:
                    if (noPAM_given and (guideSeq != offseq)) or ((noPAM_given==False) and (guideSeq != offseq+PAM)):
                        offSeqs.append(((offseq+PAM), tc, ts, te, tst))
                    else:
                        on_targets.append(((offseq+PAM), (rev_comp_seq(preseq[:4])+offseq+comp_seq(pamseq[:6])), tc, ts, te, tst))
        else:
            sys.stderr.write("WARNING: RIsearch output line is missing columns, or info in columns.\nLINE: "+line)
    inf.close()
    return  offSeqs, off_counts, on_targets

# Returns all valid Cas9 targets from Cas-OFFinder result file
def read_casoff_results(guideSeq, casoff_file, count_mms=False):
    on_targets = []
    offSeqs = []
    off_counts = {"GG":[0]*7, "AG":[0]*7, "GA":[0]*7}
    inf = gzip.open(casoff_file, 'rt') if casoff_file.endswith(".gz") else open(casoff_file, 'rt')
    x = 0
    for line in inf:
        x = x + 1
        cols = line.rstrip().split("\t")
        if line[0]!="#" and len(cols)>7 and cols[0]=="X":
            bulge_type, crRNA, DNA, chromosome, position, strand, mismatches, bulge_size = cols[:8]
            PAM = DNA.upper()[-3:]
            if PAM[1:3] in ["GG", "AG", "GA"]:
                offseq = DNA.upper()
                mm_count = int(mismatches)
                # determine and save the number of mismatches for this guide
                if count_mms:
                    if mm_count<7:
                        off_counts[PAM[1:3]][mm_count] += 1

                if mm_count<7:
                    if DNA[:len(guideSeq)]==guideSeq:
                        if strand=="+":
                            on_targets.append(((offseq), DNA, chromosome, position, str(int(position)+len(offseq)), strand))
                        else:
                            on_targets.append(((offseq), DNA, chromosome, str(int(position)-len(offseq)), position, strand))
                    else:
                        if strand=="+":
                            offSeqs.append(((offseq), chromosome, position, str(int(position)+len(offseq)), strand))
                        else:
                            offSeqs.append(((offseq), chromosome, str(int(position)-len(offseq)), position, strand))
    inf.close()
    return  offSeqs, off_counts, on_targets

# Returns all valid Cas9 targets from plain new-line seperated input off-targets file
def read_standard_offtargets_input(guideSeq, in_file, count_mms=False):
    on_targets = []
    offSeqs = []
    off_counts = {"GG":[0]*7, "AG":[0]*7, "GA":[0]*7}
    inf = gzip.open(in_file, 'rt') if in_file.endswith(".gz") else open(in_file, 'rt')
    x = 0
    for line in inf:
        cols = line.rstrip().split("\t")
        if line[0]!="#" and len(cols[0])==len(guideSeq):
            x = x + 1
            offseq = cols[0].upper()
            PAM = offseq[-3:]
            if PAM[1:3] in ["GG", "AG", "GA"]:
                mm_count = sum ( offseq[i] != guideSeq[i] for i in range(len(offseq)-3) )
                # determine and save the number of mismatches for this guide
                if count_mms:
                    if mm_count<7:
                        off_counts[PAM[1:3]][mm_count] += 1
                if mm_count<7:
                    if offseq==guideSeq:
                        on_targets.append((offseq, offseq, None, None, None, None))
                    else:
                        offSeqs.append((offseq, None, None, None, None))
    inf.close()
    return  offSeqs, off_counts, on_targets

#Read given off-target file whether its risearch casoff or standard regular new-line seperated file
def read_offtargets_file(guideSeq, offtargets_file, noPAM_given=False, count_mms=False, chromosome_names=None):
    if "risearch" in offtargets_file:
        if chromosome_names==None:
            sys.stdout.write('#RUNNING: reading given risearch output file "'+offtargets_file+'" for gRNA/on-target:'+guideSeq+'.\n')
            return read_risearch_results(guideSeq, offtargets_file, noPAM_given=noPAM_given, count_mms=count_mms, on_targets = [], offSeqs = [], off_counts = {"GG":[0]*7,"AG":[0]*7,"GA":[0]*7})
        else: # READ from multiple files
            chr_inf = gzip.open(chromosome_names, 'rt') if chromosome_names.endswith(".gz") else open(chromosome_names, 'rt')
            on_targets = []
            offSeqs = []
            off_counts = {"GG":[0]*7, "AG":[0]*7, "GA":[0]*7}
            ris_dir = "/".join(offtargets_file.split("/")[:-1])
            ris_file = offtargets_file.split("/")[-1]
            for chrid in chr_inf:
                sys.stdout.write('#RUNNING: reading given risearch output file "'+"/".join([ris_dir, chrid.rstrip(), ris_file])+'".\n')
                if os.path.isfile("/".join([ris_dir, chrid.rstrip(), ris_file])):
                    offSeqs, off_counts, on_targets = read_risearch_results(guideSeq, "/".join([ris_dir, chrid.rstrip(), ris_file]), noPAM_given=noPAM_given, count_mms=count_mms, on_targets = on_targets, offSeqs = offSeqs, off_counts = off_counts)
            chr_inf.close()
            return offSeqs, off_counts, on_targets
    else:
        inf = gzip.open(offtargets_file, 'rt') if offtargets_file.endswith(".gz") else open(offtargets_file, 'rt')
        casoff = True if inf.readline().startswith("#Bulge") else False
        inf.close()
        if casoff:
            sys.stdout.write('#RUNNING: reading given Cas-OFFinder file "'+offtargets_file+'".\n')
            return read_casoff_results(guideSeq, offtargets_file, count_mms=count_mms)
        else:
            sys.stdout.write('#RUNNING: reading given offtarget file "'+offtargets_file+'" as regular off-target file.\n')
            return read_standard_offtargets_input(guideSeq, offtargets_file, count_mms=count_mms)

# Read guide sequences from fasta file
def read_guides_fasta(fasta_file):
    ontargets={}
    for record in SeqIO.parse(fasta_file, "fasta"):
        seq = str(record.seq).upper()
        if len(str(record.seq))==23:
            ontargets[record.id] = seq
        else:
            # get valid guides with PAM NGG and 23nt
            x=0
            grna_seq=""
            for i in range(len(seq)):
                if seq[i] not in ["A", "C", "G", "T", "U"]:
                    grna_seq =""
                elif len(grna_seq)<22:
                    grna_seq = grna_seq + seq[i]
                else:
                    grna_seq = grna_seq + seq[i]
                    if grna_seq[-2:] == "GG":
                        x+=1
                        ontargets[record.id+"_gRNA_"+str(x)] = grna_seq
                    grna_seq = grna_seq[1:]
            revseq = rev_comp_seq(seq)
            for i in range(len(revseq)):
                if revseq[i] not in ["A", "C", "G", "T", "U"]:
                    grna_seq =""
                elif len(grna_seq)<22:
                    grna_seq = grna_seq + revseq[i]
                else:
                    grna_seq = grna_seq + revseq[i]
                    if grna_seq[-2:] == "GG":
                        x+=1
                        ontargets[record.id+"_rev_gRNA_"+str(x)] = grna_seq
                    grna_seq = grna_seq[1:]

            if x==1:
                ontargets[record.id] = ontargets[record.id+"_gRNA_"+str(x)]
                del ontargets[record.id+"_gRNA_"+str(x)]
    return ontargets

## Summarize the features for given on-targets
def get_energy_features_for_guides(guideSeqs):
    feature_out = {}
    for guide, guideSeq in guideSeqs.items():
        feature_out[guide]={}
        feature_out[guide]["RNA_DNA_eng"] = -1.00000 * get_eng(guideSeq, guideSeq, calcRNADNAenergy, GU_allowed=False, pos_weight=False, pam_corr=False, grna_folding=False, dna_opening=False, dna_pos_wgh=False)
        feature_out[guide]["RNA_DNA_eng_weighted"] = -1.00000 * get_eng(guideSeq, guideSeq, calcRNADNAenergy, GU_allowed=False, pos_weight=True, pam_corr=False, grna_folding=False, dna_opening=False, dna_pos_wgh=False)
        feature_out[guide]["DNA_DNA_opening"] = -1.00000 * sum(calcDNAopeningScore(guideSeq))
        feature_out[guide]["spacer_self_fold"] = get_rnafold_eng(guideSeq[:20])
        feature_out[guide]["CRISPRoff_score"] = get_eng(guideSeq, guideSeq, calcRNADNAenergy, GU_allowed=False, pos_weight=True, pam_corr=True, grna_folding=True, dna_opening=True, dna_pos_wgh=False)
    features=["RNA_DNA_eng", "RNA_DNA_eng_weighted", "DNA_DNA_opening", "spacer_self_fold", "CRISPRoff_score"]
    return features, feature_out

## Summarize the on off target info for given on_targets

In [3]:
# 如何从30mers生成23mers?
PRE_GUIDE=4
GUIDE=20
PRE_PAM=1
PAM='GG'
nPAM=len(PAM)
POST_PAM=3
TOTAL=PRE_GUIDE+GUIDE+PRE_PAM+nPAM+POST_PAM
PAMoffset=PRE_GUIDE+GUIDE+PRE_PAM
#23mers = v[1][PRE_GUIDE:TOTAL-POST_PAM]

In [4]:
def read_30mers_fasta(fn):
    fasta = SeqIO.parse(fn, "fasta")
    d = OrderedDict()
    for r in fasta:
        id, seq = r.id, r.seq
        if id in d:
            print('sequence identifiers most be unique', file=sys.stderr)
            sys.exit(1)
        elif not len(seq) == eLENGTH:
            print('"%s" is not a sequence of length %d' % (seq, eLENGTH), file=sys.stderr)
            sys.exit(1)
        else:
            d[id] = list()
            d[id].append(seq)
    return d

def get_23_mers(d):
    d_23mers  = {}
    for k, v in d.items():
        d_23mers[k] = str(v[0][PRE_GUIDE:TOTAL-POST_PAM])
    return d_23mers

## Components of CRISPRon

In [5]:
np.set_printoptions(threshold=np.inf)

#length of input seq
eLENGTH=30
#depth of onehot encoding
eDEPTH=4

def onehot(x):
    z = list()
    for y in list(x):
        if y in "Aa":  z.append(0)
        elif y in "Cc": z.append(1)
        elif y in "Gg": z.append(2)
        elif y in "TtUu": z.append(3)
        else:
            print("Non-ATGCU character " + data[l], file=sys.stderr)
            raise Exception
    return z

def set_data(DX, s):
    for j,x in enumerate(onehot(s)):
        DX[j][x] = 1

def preprocess_seq(data):
    dkeys = data.keys()
    DATA_X = np.zeros((len(dkeys),eLENGTH,eDEPTH), dtype=np.float32) # onehot
    DATA_G = np.zeros((len(dkeys)), dtype=np.float32) #deltaGb

    seqs = list()

    for l, id in enumerate(dkeys):
        d = data[id]
        set_data(DATA_X[l], d[1])
        DATA_G[l] = d[0]
        seqs.append(d[1])

    return (seqs, DATA_X, DATA_G,)

# main function of get_deltaG()

In [6]:
# Only need deltaG
args_duplex_energy_params = "../data/model/energy_dics.pkl"
read_energy_parameters(args_duplex_energy_params)
def CRISPRspec_get_deltaG(guideSeqs,test_mode=False):
    # Pre_STEP 1: Setting arguments and energy parameters.    
    args_rnafold_x = 'RNAfold'
    if test_mode:
        sys.stdout.write('#STEP 1: arguments and energy parameters are ready.\n')
    global RNAFOLD_EXE
    RNAFOLD_EXE = args_rnafold_x

    # Pre_STEP 2: Check input guide sequences.
    if len(guideSeqs.keys())==0:
        sys.stdout.write('#ABORTED: No guide sequence given.\n')
        sys.stdout.write('#ERROR: There are no guide sequences given as input.\n')
        exit()

    # STEP 3: Get on-target energy features 
    feature_names, feature_values_dic = get_energy_features_for_guides(guideSeqs)
    deltaG_result=pd.DataFrame(columns=['guideID', 'guideSeq']+feature_names)
    for guide, feature_dic in feature_values_dic.items():
            tmp=[guide, guideSeqs[guide]]+[str(feature_dic[feature]) for feature in feature_names]
            #deltaG_result = deltaG_result.append(tmp,ignore_index= True)
            deltaG_result.loc[len(deltaG_result)] = tmp

    # FINISH
    if test_mode:
        sys.stdout.write('#END: FINISHED RUNNING THE PIPELINE WITH NO ERROR.\n')
        sys.stdout.write('#CRISPRspec_pipeline finished.\n')

    return deltaG_result

# main function of get_on_score()

In [7]:
def get_on_score(d,MODELD):
    # id, deltaGb, seq
    (s, x, g) = preprocess_seq(d)
    tinput = list()
    tinput.append(x)
    tinput.append(g)

    #evaluate per model
    v = list()
    for i, md in enumerate(MODELD):
        model = models.load_model(md)
        model.compile(loss='mse', metrics=['mae', 'mse'])
        predicted = model.predict(tinput, use_multiprocessing=True, workers=32)
        predicted = [p[0] for p in predicted]
        v.append([list(d.keys()), s, predicted])

    #average model outputs
    ids = v[0][0]
    sequences = v[0][1]
    predicted = [0 for i in range(len(sequences))]
    nmodels = len(MODELD)
    for m in range(nmodels):
        predicted = [predicted[i] + v[m][2][i]/nmodels for i in range(len(sequences))]

    result=pd.DataFrame({'ID':ids,'30mer_sequences':sequences,'On_Scores':predicted})
    return result

In [8]:
def seqs_add_deltaG(d, crisproff_df):    
    for index, row in crisproff_df.iterrows():
        id, g = row[0], row[6]
        try:
            g = float(g)
        except:
            sys.print("ERROR: deltaG value is not float: "+g+"\n")
            sys.exit(1)
        d[id].insert(0, g)
    return d

# Execution:Predict On_target score

In [9]:
def CRISPROn_score_pipeline(d):
    #For compute CRISPRon scores
    MODELD=glob.glob('../data/deep_models/best/*/') #paths to model(s) #$DATADIR/deep_models/best/*/

    #Get seqs: 30mers-> 23mers
    guideSeqs = get_23_mers(d)

    #Get deltaG
    deltaG_result= CRISPRspec_get_deltaG(guideSeqs=guideSeqs,test_mode=False)
    data = seqs_add_deltaG(d, deltaG_result)
    result=get_on_score(data,MODELD)
    return result

In [10]:
fasta_path = '../test/outdir.test/30mers.fa'
seq_data = read_30mers_fasta(fn=fasta_path)

In [11]:
result=CRISPROn_score_pipeline(seq_data)
result

11/11 [==============================] - 2s 92ms/step


,ID,30mer_sequences,On_Scores
0,test_seq_p_19,"(T, G, C, C, A, T, G, C, A, A, G, G, C, A, A, ...",50.426987
1,test_seq_p_22,"(C, A, T, G, C, A, A, G, G, C, A, A, A, C, T, ...",49.218208
2,test_seq_p_23,"(A, T, G, C, A, A, G, G, C, A, A, A, C, T, G, ...",49.222065
3,test_seq_p_24,"(T, G, C, A, A, G, G, C, A, A, A, C, T, G, C, ...",60.418915
4,test_seq_p_25,"(G, C, A, A, G, G, C, A, A, A, C, T, G, C, T, ...",53.734760
...,...,...,...
336,test_seq2_m_62,"(C, A, G, T, T, C, T, G, T, G, C, T, T, C, C, ...",13.416720
337,test_seq2_m_28,"(G, T, G, A, C, T, A, T, T, G, T, A, C, T, T, ...",33.848930
338,test_seq2_m_9,"(T, G, T, A, G, A, G, G, A, G, A, G, A, A, A, ...",58.765717
339,test_seq2_m_8,"(G, T, A, G, A, G, G, A, G, A, G, A, A, A, A, ...",43.916271


# 遗传算法

In [12]:
print(PRE_GUIDE,TOTAL-POST_PAM)
tmp_seq="TTGTGCATTTGTGACAAGTCCTAGTGGGAT"
print(tmp_seq[PRE_GUIDE:TOTAL-POST_PAM])
print(tmp_seq[4:27])
print(tmp_seq[25:27])

4 27
GCATTTGTGACAAGTCCTAGTGG
GCATTTGTGACAAGTCCTAGTGG
GG


序列长度30，[25,26]两个位置一定为'GG',剩下28个自由位置，每个位置四种可能。

In [26]:
def IntArray2ATCG(integer_array):
    mapping = {0: 'A', 1: 'T', 2: 'C', 3: 'G'}
    ATCG_array = [mapping[i] for i in integer_array]
    return ''.join(ATCG_array) #str

In [43]:
x = np.ones(28)
seq28 = IntArray2ATCG(x)
seq30 = seq28[0:25] + "GG" + seq28[25:]
print(seq30)
seq = OrderedDict()
seq['test'] = [seq30]
tmp=CRISPROn_score_pipeline(seq).iloc[0,2]

TTTTTTTTTTTTTTTTTTTTTTTTTGGTTT


1/1 [==============================] - 0s 215ms/step


In [45]:
import numpy as np
def schaffer(p):
    '''
    This function has plenty of local minimum, with strong shocks
    global minimum at (0,0) with value 0
    https://en.wikipedia.org/wiki/Test_functions_for_optimization
    '''
    x = np.array(p)
    seq28 = IntArray2ATCG(x)
    seq30 = seq28[0:25] + "GG" + seq28[26:]
    seq['test'] = [seq30]
    return CRISPROn_score_pipeline(seq).iloc[0,2]


# %%
from sko.GA import GA

n=28
# default iter=800
ga = GA(func=schaffer, n_dim=28, size_pop=50, max_iter=50, prob_mut=0.001, lb=[0]*n, ub=[3]*n, precision=[1]*n)
best_x, best_y = ga.run()
print('best_x:', best_x, '\n', 'best_y:', best_y)
# %% Plot the result
import pandas as pd
import matplotlib.pyplot as plt

Y_history = pd.DataFrame(ga.all_history_Y)
print(Y_history)
fig, ax = plt.subplots(2, 1)
ax[0].plot(Y_history.index, Y_history.values, '.', color='red')
Y_history.min(axis=1).cummin().plot(kind='line')
plt.show()

1/1 [==============================] - 0s 311ms/step


KeyboardInterrupt: 